# Analyse job offer text

In [1]:
import nltk
from bs4 import BeautifulSoup
import re
import pandas as pd
import string

## Extracting text from HTML

In [21]:
# !pip install selenium
from selenium import webdriver 
import os
from selenium.webdriver.common.by import By

url = "https://www.linkedin.com/jobs/view/1732636707/"
# url = 'https://www.linkedin.com/jobs/view/1489206302/'

PROJECT_ROOT = os.path.abspath(os.path.dirname('__file__'))
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")

driver = webdriver.Chrome(executable_path = DRIVER_BIN)

driver.get(url)

driver.implicitly_wait(20) # seconds
description = driver.find_element_by_class_name('description')



elementHTML = description.get_attribute('innerHTML')
elementHTML = BeautifulSoup(elementHTML)

text = elementHTML.get_text()
text = text.replace('\n', ' ')
text = text.replace('(', ' ')
text = text.replace(')', ' ')
text = text.replace('/', ' ')
text = re.sub(' +',' ', text)



#input('Press ENTER to close the automated browser')

driver.quit()
print('done')
print('---------------------------------------------')
print('---------------Job Description---------------')
print('---------------------------------------------')
print(text)

done
---------------------------------------------
---------------Job Description---------------
---------------------------------------------
100%, Zurich, temporaryThe Chair of Systems Design at ETH Zurich invites applications for two PhD positions in the field of Computational Social Sciences. The first PhD position focuses on the analysis of political speeches using NLP approaches. The second PhD position focuses on quantifying resilience in collaboration networks. Both PhD positions leverage a new data set on the Swiss parliament longitudinal, 1891 until today . The two positions are associated with our SNF project Analyzing Co-Sponsorship Networks from 127 Years of the Swiss Federal Assembly , but successful candidates are required to develop their own research plan. Project background The two PhD positions are loosely associated with our SNF project. In this project we aim to detect when politicians change their opinions and why. We examine parliamentary speeches using both supe

In [3]:
from selenium.webdriver.common.by import By
# url = "https://www.linkedin.com/jobs/view/1732636707/"
url = 'https://www.linkedin.com/jobs/view/1489206302/'

PROJECT_ROOT = os.path.abspath(os.path.dirname('__file__'))
DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")

driver = webdriver.Chrome(executable_path = DRIVER_BIN)

driver.get(url)


driver.implicitly_wait(20) # seconds
title = driver.find_element_by_class_name('topcard').find_element(By.XPATH, '//h1[contains(@class, "topcard__title")]')
company = driver.find_element_by_class_name('topcard').find_element(By.XPATH, '//span[contains(@class, "topcard__flavor")]')
location = driver.find_element_by_class_name('topcard').find_element(By.XPATH, '//span[contains(@class, "topcard__flavor topcard__flavor--bullet")]')
time = driver.find_element_by_class_name('topcard').find_element(By.XPATH, '//span[contains(@class, "topcard__flavor--metadata posted-time-ago__text")]')
num_app = driver.find_element_by_class_name('topcard').find_element(By.XPATH, '//figcaption[contains(@class, "num-applicants__captio")]')

description = driver.find_element_by_class_name('description').find_element(By.XPATH, '//div[contains(@class, "description__text description__text--rich")]')
job_criteria = driver.find_element_by_class_name('description').find_element(By.XPATH, '//ul[contains(@class, "job-criteria__list")]')

data = dict()
data.update({'title' : title , 'company' : company , 'location' : location , 'time' : time 
             , 'num_app' : num_app ,'description' : description , 'job_criteria' : job_criteria })

linkedIn = dict()

for key, value in data.items():
    elementHTML = value.get_attribute('innerHTML')
    elementHTML = BeautifulSoup(elementHTML)
    text = elementHTML.get_text()
    text = text.replace('\n', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('/', ' ')
    text = re.sub(' +',' ', text)
    linkedIn[key] = text



input('Press ENTER to close the automated browser')

driver.quit()
print('done')
# print('---------------------------------------------')
# print('---------------Job Description---------------')
# print('---------------------------------------------')
for key, value in linkedIn.items():
    print('\n')
    print('---------------{}---------------'.format(key))
    print('\n')
    print(value)

Press ENTER to close the automated browser
done


---------------title---------------


Machine Learning Scientist


---------------company---------------


ASOS.com


---------------location---------------


London, England, United Kingdom


---------------time---------------


2 weeks ago


---------------num_app---------------


Over 200 applicants


---------------description---------------


ASOS is one of the UK’s top fashion and beauty destinations, expanding globally at a rapid pace. Our values are to be authentic, brave and creative, and we live and breathe these in everything we do.We believe fashion can make you look, feel and be your best and, with technology in our DNA, we deliver the latest trends to our digital-obsessed 20-something market. Our award winning Tech teams sit at the heart of our business. We deliver technical innovation and pioneer incredible solutions, which are crucial to our continued success. We’re extremely ambitious and thrive on the individuality of 

## Frequency Analysis

In [4]:
#download nltk attribute if needed
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('reuters')

[nltk_data] Downloading package reuters to C:\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

#### tokenization

In [5]:
filtered_text = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

#### stopwords

In [6]:
from nltk.corpus import stopwords

filtered_text = [word for word in filtered_text if word not in stopwords.words('english')]

#### punctuation

In [7]:
filtered_text = [''.join(c for c in s if c not in string.punctuation) for s in filtered_text]
filtered_text = [s for s in filtered_text if s]

#### lowercase

In [8]:
filtered_text = [word.lower() for word in filtered_text]

#### Word Stemming

In [9]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
filtered_text_stem = [stemmer.stem(t) for t in filtered_text]

#### Lemmatization

In [10]:
lemmatizer = nltk.WordNetLemmatizer()

filtered_text_lem = [lemmatizer.lemmatize(t) for t in filtered_text]

#### NLTK Frequency Distributions

In [11]:
#nltkText = [filtered_text_stem, filtered_text_lem]

fdist = nltk.FreqDist(filtered_text_lem)

fdistDF = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
fdistDF = fdistDF.rename(columns={'index':'word', 0:'count'})
fdistDF = fdistDF.sort_values(by='count', ascending=False)
print('-----------------head words-----------------')
print(fdistDF.head(10).reset_index(drop=True))
print('-----------------tail words-----------------')
print(fdistDF.tail(10).reset_index(drop=True))

-----------------head words-----------------
                             word  count
0                       seniority      1
1                        levelnot      1
2            applicableemployment      1
3                 typefulltimejob      1
4  functionotherindustriesapparel      1
5           fashioninternetretail      1
-----------------tail words-----------------
                             word  count
0                       seniority      1
1                        levelnot      1
2            applicableemployment      1
3                 typefulltimejob      1
4  functionotherindustriesapparel      1
5           fashioninternetretail      1


## Analyze document words with data science job descripiton database

To indentify what skills are specific for this job vacancy in particulair we have to create a data science job description database. Using this database we can identify what is specific about this job with the steps below.

#### Building a Term-Document Matrix

Use scikit-learn's <code>TfidfVectorizer</code> class to construct a [term-document matrix](http://en.wikipedia.org/wiki/Document-term_matrix) containing the TF-IDF score for each word in each document in the data science job description database. In essence, the rows of this sparse matrix correspond to documents in the corpus, the columns represent each word in the vocabulary of the corpus, and each cell contains the TF-IDF value for a given word in a given document.

#### TF-IDF Scores

Now that we've built the term-document matrix, we can explore its contents:

In [12]:
from random import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf = TfidfVectorizer(stop_words='english', analyzer='word', lowercase=True)
content = text.split()
tfidf_vector = tfidf.fit_transform(content)

feature_names = tfidf.get_feature_names()

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content)

tdm= pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
print ('TDM contains ' + str(len(feature_names)) + ' terms and ' + str(tdm.shape[0]) + ' documents')

print ('first term: ' + feature_names[0])
print ('last term: ' + feature_names[len(feature_names) - 1])

for i in range(0, 4):
    print ('random term: ' + feature_names[randint(1,len(feature_names) - 2)])

TDM contains 7 terms and 7 documents
first term: applicableemployment
last term: typefull
random term: levelnot
random term: timejob
random term: timejob
random term: fashioninternetretail


#### Generating the Summary

That's all we'll need to produce a summary for any document in the corpus. In the example code below, we start by randomly selecting an article from the data science job description database. We iterate through the article, calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. We normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). Then we sort the sentences by their scores, and return the highest-scoring sentences as our summary. The number of sentences returned corresponds to roughly 20% of the overall length of the article.

In [23]:
from __future__ import division
from nltk.corpus import reuters
import math

def tokenize_and_stem(sentence):
    filtered_text = [word for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    stemmer = SnowballStemmer("english")
    filtered_text_stem = [stemmer.stem(t) for t in filtered_text]
    return filtered_text_stem
    
    
    
article_id = randint(0, tdm.shape[0] - 1)
article_text = reuters.raw(reuters.fileids()[article_id])
# article_text = "100%, Zurich, temporaryThe Chair of Systems Design at ETH Zurich invites applications for two PhD positions in the field of Computational Social Sciences. The first PhD position focuses on the analysis of political speeches using NLP approaches. The second PhD position focuses on quantifying resilience in collaboration networks. Both PhD positions leverage a new data set on the Swiss parliament longitudinal, 1891 until today . The two positions are associated with our SNF project Analyzing Co-Sponsorship Networks from 127 Years of the Swiss Federal Assembly , but successful candidates are required to develop their own research plan. Project background The two PhD positions are loosely associated with our SNF project. In this project we aim to detect when politicians change their opinions and why. We examine parliamentary speeches using both supervised and unsupervised ML approaches and the temporal dynamics of how members of parliament work together. Our project builds on a newly compiled data set of the Swiss parliament from 1891-today. It contains all speeches, political interactions and collaboration networks from members of parliament over the past 128 years. For more information, see .Job DescriptionOur multi-lingual data set offers the opportunity to develop new methodologies in the field of NLP including e.g., analyzing evolving speech patterns, speech complexity and sentiment analysis and network science including e.g., analyzing collaboration networks, multi-level networks, developing inferential network models . Starting from this unique data set, prospective candidates will develop their own research interests. The positions are available from May 2020 flexible and are fully funded for 3+ years by the Swiss National Science Foundation SNF . Deadline for applications is February 16, 2020. Your profile PhD Position With Focus On NLP MSc degree in computer science or related field Strong analytical skills and background in natural language processing As most of the data is in German, good German skills are required PhD Position With Focus On Data Science And Networks MSc degree in physics, data science, computer science alternatively MSc MA in social science with exceptional skills in quantitative analysis strong background in data analysis and modelling excellent programming skills and ideally also high-performance computing and machine learning database management e.g., SQL, neo4j As most of the data is in German, good German skills are required Apart from excellent technical skills, both candidates should show an interest in social and political science questions, be able to communicate well in an interdisciplinary team, and be highly motivated and creative in what they do. ETH Zurich ETH Zurich is one of the world’s leading universities specialising in science and technology. We are renowned for our excellent education, cutting-edge fundamental research and direct transfer of new knowledge into society. Over 30,000 people from more than 120 countries find our university to be a place that promotes independent thinking and an environment that inspires excellence. Located in the heart of Europe, yet forging connections all over the world, we work together to develop solutions for the global challenges of today and tomorrow. Interested?We look forward to receiving your online application with the following documents: CV, motivation letter including detailed description of relevant skills and your interest in the above mentioned research project and transcript of grades. Please apply exclusively via the online application portal. Applications sent by email or post will not be considered.Further information about the Chair of Systems Design can be found on our website . For specific inquiries, please contact Prof. Dr. Frank Schweitzer at click apply no applications .Seniority levelAssociateEmployment typeTemporaryJob functionResearchAnalystInformation TechnologyIndustriesInformation Technology and ServicesResearchStaffing and Recruiting"

sent_scores = []
for sentence in nltk.sent_tokenize(article_text):
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / 5))
sent_scores.sort(key=lambda sent: sent[0], reverse=True)

print ('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    print (summary_sentence[1])

print ('\n*** ORIGINAL ***')
print (article_text)

*** SUMMARY ***
THAI TRADE DEFICIT WIDENS IN FIRST QUARTER
  Thailand's trade deficit widened to 4.5
  billion baht in the first quarter of 1987 from 2.1 billion a
  year ago, the Business Economics Department said.
It said Janunary/March imports rose to 65.1 billion baht
  from 58.7 billion.

*** ORIGINAL ***
THAI TRADE DEFICIT WIDENS IN FIRST QUARTER
  Thailand's trade deficit widened to 4.5
  billion baht in the first quarter of 1987 from 2.1 billion a
  year ago, the Business Economics Department said.
      It said Janunary/March imports rose to 65.1 billion baht
  from 58.7 billion. Thailand's improved business climate this
  year resulted in a 27 pct increase in imports of raw materials
  and semi-finished products.
      The country's oil import bill, however, fell 23 pct in the
  first quarter due to lower oil prices.
      The department said first quarter exports expanded to 60.6
  billion baht from 56.6 billion.
      Export growth was smaller than expected due to lower
  e